# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [31]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore). Don't forget to enrich the table with the 'dayofweek' column from the previous day's .csv-file.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [4]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')
y = pd.read_csv('../data/dayofweek.csv')
df.head()

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
X = df
y = y['dayOfWeek']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [7]:
svc = SVC(random_state=21, probability=True)

In [8]:
param_grid = {'C': [0.01, 0.1, 1, 1.5, 5, 10],
              'kernel': ['linear', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto'],
              'class_weight': ['balanced', None]
              }

gs = GridSearchCV(svc, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}
лучший score: 0.8761090458488228


## 📊 Пояснение к `cv_results_` в `GridSearchCV`

После `GridSearchCV.fit()` можно получить:
```python
results = pd.DataFrame(gs.cv_results_)
```
Он содержит **подробные метрики для КАЖДОЙ комбинации параметров**, которые пробовал `GridSearchCV`.

---

### ✅ Основные столбцы в `cv_results_`:

| Колонка | Что значит |
|---------|------------|
| **mean_fit_time** | Среднее время обучения модели на одной комбинации параметров по всем фолдам. |
| **std_fit_time** | Стандартное отклонение времени обучения по фолдам. |
| **mean_score_time** | Среднее время вычисления предсказаний на валидации по фолдам. |
| **std_score_time** | Стандартное отклонение времени предсказаний по фолдам. |
| **param_C, param_kernel, param_gamma, param_class_weight** | Значения параметров модели, использованных в этой комбинации. |
| **params** | Словарь со всеми параметрами для удобства. |
| **split0_test_score ... split4_test_score** | Accuracy модели на каждом из 5 фолдов при этой комбинации параметров. |
| **mean_test_score** | Среднее значение accuracy по всем фолдам для этой комбинации параметров. |
| **std_test_score** | Стандартное отклонение accuracy по всем фолдам (оценка стабильности). |
| **rank_test_score** | Ранг комбинации параметров (1 = лучший `mean_test_score`). |

---

### ✅ Формулы:

#### 1️⃣ Среднее время обучения:
$$
\text{mean\_fit\_time} = \frac{\sum_{i=1}^{k} t_{fit}^{(i)}}{k}
$$
где \( k = \text{количество фолдов} \).

---

#### 2️⃣ Accuracy на фолде:
$$
\text{split}_j\_\text{test\_score} = \frac{\text{количество верных предсказаний на фолде } j}{\text{количество примеров на фолде } j}
$$

---

#### 3️⃣ Среднее accuracy:
$$
\text{mean\_test\_score} = \frac{\sum_{j=1}^{k} \text{split}_j\_\text{test\_score}}{k}
$$

---

#### 4️⃣ Стандартное отклонение accuracy:
$$
\text{std\_test\_score} = \sqrt{ \frac{ \sum_{j=1}^{k} (\text{split}_j\_\text{test\_score} - \text{mean\_test\_score})^2 }{k} }
$$

---

#### 5️⃣ Ранг:
- Присваивается на основе сортировки `mean_test_score` по убыванию.
- `rank_test_score = 1` означает **лучший результат среди всех комбинаций параметров**.

---

## ✅ Как использовать `cv_results_`:
- Сортируй по `rank_test_score`, чтобы найти лучшие комбинации.
- Смотри `mean_test_score` и `std_test_score`, чтобы оценить **качество и стабильность.**
- Анализируй параметры (`param_*`), чтобы понять, какие настройки дают лучший результат.
- Используй `mean_fit_time` для выбора между качеством и скоростью, если модель медленная.

---

## 🎯 Ключ для анализа:
- **Если простая модель даёт сопоставимый `mean_test_score` с более сложной, лучше взять простую модель.**
- **Низкий `std_test_score` говорит о стабильности модели при кросс-валидации.**
- Используй `cv_results_` для **системного и аргументированного выбора модели в проектах.**

---

In [15]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
21,0.009319,0.000186,0.004001,0.001892,balanced,gini,22,"{'class_weight': 'balanced', 'criterion': 'gin...",0.885185,0.862963,0.903704,0.881041,0.832714,0.873121,0.023998,1
20,0.012379,0.003819,0.004445,0.000524,balanced,gini,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.859259,0.903704,0.884758,0.828996,0.873121,0.026300,2
22,0.010527,0.001882,0.004377,0.000554,balanced,gini,23,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
23,0.010583,0.002553,0.003665,0.000235,balanced,gini,24,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
41,0.016805,0.010670,0.006868,0.006089,balanced,gini,42,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [10]:
tree = DecisionTreeClassifier(random_state=21)

In [11]:
param_grid = {'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None]
            }

gs = GridSearchCV(tree, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22}
лучший score: 0.8731212997384002


In [19]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
21,0.009319,0.000186,0.004001,0.001892,balanced,gini,22,"{'class_weight': 'balanced', 'criterion': 'gin...",0.885185,0.862963,0.903704,0.881041,0.832714,0.873121,0.023998,1
20,0.012379,0.003819,0.004445,0.000524,balanced,gini,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.859259,0.903704,0.884758,0.828996,0.873121,0.026300,2
22,0.010527,0.001882,0.004377,0.000554,balanced,gini,23,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
23,0.010583,0.002553,0.003665,0.000235,balanced,gini,24,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
41,0.016805,0.010670,0.006868,0.006089,balanced,gini,42,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [20]:
frt = RandomForestClassifier(random_state=21)

In [21]:
param_grid = {'n_estimators': [5, 10, 50, 100],
              'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None],
              'random_state': [21]}

gs = GridSearchCV(frt, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50, 'random_state': 21}
лучший score: 0.9042902381935839


In [22]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_n_estimators,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
502,0.130881,0.020035,0.008854,0.003382,None,gini,28,50,21,"{'class_weight': None, 'criterion': 'gini', 'm...",0.922222,0.900000,0.907407,0.903346,0.888476,0.904290,0.010961,1
515,0.272242,0.071102,0.012810,0.002437,None,gini,31,100,21,"{'class_weight': None, 'criterion': 'gini', 'm...",0.918519,0.911111,0.900000,0.910781,0.877323,0.903547,0.014380,2
118,0.122951,0.012716,0.007367,0.000363,balanced,gini,30,50,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.881481,0.907063,0.895911,0.902817,0.013554,3
134,0.119102,0.001749,0.009486,0.000807,balanced,gini,34,50,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.892593,0.907063,0.884758,0.902809,0.013010,4
543,0.221888,0.008617,0.011343,0.000632,None,gini,38,100,21,"{'class_weight': None, 'criterion': 'gini', 'm...",0.914815,0.911111,0.900000,0.903346,0.884758,0.902806,0.010460,5


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [24]:
params = {
    'n_estimators': [5, 10, 50, 100],
    'max_depth': np.arange(1, 50),
    'class_weight': [None, 'balanced'],
    'criterion': ['gini', 'entropy'],
    'random_state': [21],
}

params_list = list(ParameterGrid(params))
data = []

for params_model in tqdm(params_list):
    estimator = RandomForestClassifier(**params_model)
    cvs = cross_val_score(estimator, X_train, y_train, cv=5, n_jobs=-1)
    info = {**params_model, 'mean_accuracy': cvs.mean(), 'std_accuracy': cvs.std()}
    data.append(info)

100%|██████████| 784/784 [01:09<00:00, 11.28it/s]


In [27]:
results =pd.DataFrame(data)
results.sort_values('mean_accuracy', ascending=False).head(10)

,class_weight,criterion,max_depth,n_estimators,random_state,mean_accuracy,std_accuracy
110,None,gini,28,50,21,0.904290,0.010961
123,None,gini,31,100,21,0.903547,0.014380
510,balanced,gini,30,50,21,0.902817,0.013554
526,balanced,gini,34,50,21,0.902809,0.013010
175,None,gini,44,100,21,0.902806,0.010460
179,None,gini,45,100,21,0.902806,0.010460
159,None,gini,40,100,21,0.902806,0.010460
183,None,gini,46,100,21,0.902806,0.010460
155,None,gini,39,100,21,0.902806,0.010460
163,None,gini,41,100,21,0.902806,0.010460


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [29]:
rfc = RandomForestClassifier(n_estimators=50, max_depth=28, criterion='gini', class_weight=None, random_state=21)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [32]:
accuracy_score(y_test, y_pred)

0.9289940828402367